# SNAPC4
Variants in SNAPC4 are associated with [Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction](https://omim.org/entry/620515). Data derived from
[Frost FG, et al. Bi-allelic SNAPC4 variants dysregulate global alternative splicing and lead to neuroregression and progressive spastic paraparesis. Am J Hum Genet. 2023](https://pubmed.ncbi.nlm.nih.gov/36965478/)

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.56


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/SNAPC4_Frost_2023.xlsx")
df.head() 

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Motor regression,Spasticity,Spastic tetraparesis,Dystonia,Dysarthria,Drooling,Gait disturbance,Hyperreflexia,Cerebral atrophy,Cerebellar atrophy
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0033044,HP:0001257,HP:0001285,HP:0001332,HP:0001260,HP:0002307,HP:0001288,HP:0001347,HP:0002059,HP:0001272
1,PMID:36965478,Bi-allelic SNAPC4 variants dysregulate global alternative splicing and lead to neuroregression and progressive spastic paraparesis,Individual 1,NaN,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",NM_003086.4,c.1436T>C,c.472-2_472-1del,NaN,...,P1Y3M,observed,observed,observed,observed,excluded,observed,observed,observed,excluded
2,PMID:36965478,Bi-allelic SNAPC4 variants dysregulate global alternative splicing and lead to neuroregression and progressive spastic paraparesis,Individual 2,NaN,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",NM_003086.4,c.595G>A,c.1325+1G>T,NaN,...,P1Y3M,observed,observed,observed,excluded,observed,observed,observed,excluded,observed
3,PMID:36965478,Bi-allelic SNAPC4 variants dysregulate global alternative splicing and lead to neuroregression and progressive spastic paraparesis,Individual 3,NaN,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",NM_003086.4,c.1321G>A,c.1157A>G,NaN,...,P2Y6M,observed,observed,observed,observed,observed,observed,observed,na,na
4,PMID:36965478,Bi-allelic SNAPC4 variants dysregulate global alternative splicing and lead to neuroregression and progressive spastic paraparesis,Individual 4,NaN,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",NM_003086.4,c.1321G>A,c.1157A>G,NaN,...,P3Y,observed,observed,observed,observed,observed,observed,observed,excluded,excluded


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 27 fields


In [5]:
SNAPC4_transcript = "NM_003086.4"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="SNAPC4",
                          transcript=SNAPC4_transcript,
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

[INFO] encoding variant "c.472-2_472-1del"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003086.4%3Ac.472-2_472-1del/NM_003086.4?content-type=application%2Fjson


In [6]:
vmanager.to_summary()


,status,count,alleles
0,mapped,13,"c.553A>G, c.2428C>T, c.1889_1899del, c.1321G>A, c.1157A>G, c.2317+1G>T, c.2900G>T, c.1436T>C, c.2527+1G>A, c.737+5G>T, c.595G>A, c.1325+1G>T, c.472-2_472-1del"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)

In [8]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,6


In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Individual 1 (FEMALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.1436T>C (heterozygous) NM_003086.4:c.472-2_472-1del (heterozygous),P1Y3M: Cognitive regression (HP:0034332); Motor regression (HP:0033044)Short stature (HP:0004322); Microcephaly (HP:0000252); Spastic tetraparesis (HP:0001285); Dystonia (HP:0001332); Dysarthria (HP:0001260); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347); Cerebral atrophy (HP:0002059); excluded: Drooling (HP:0002307); excluded: Cerebellar atrophy (HP:0001272)
Individual 2 (FEMALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.595G>A (heterozygous) NM_003086.4:c.1325+1G>T (heterozygous),P1Y3M: Cognitive regression (HP:0034332); Motor regression (HP:0033044)Short stature (HP:0004322); Spastic tetraparesis (HP:0001285); Dystonia (HP:0001332); Drooling (HP:0002307); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347); Cerebellar atrophy (HP:0001272); excluded: Microcephaly (HP:0000252); excluded: Dysarthria (HP:0001260); excluded: Cerebral atrophy (HP:0002059)
Individual 3 (MALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.1321G>A (heterozygous) NM_003086.4:c.1157A>G (heterozygous),P2Y6M: Cognitive regression (HP:0034332); Motor regression (HP:0033044)Short stature (HP:0004322); Microcephaly (HP:0000252); Spastic tetraparesis (HP:0001285); Dystonia (HP:0001332); Dysarthria (HP:0001260); Drooling (HP:0002307); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347)
Individual 4 (MALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.1321G>A (heterozygous) NM_003086.4:c.1157A>G (heterozygous),P3Y: Cognitive regression (HP:0034332); Motor regression (HP:0033044)Short stature (HP:0004322); Microcephaly (HP:0000252); Spastic tetraparesis (HP:0001285); Dystonia (HP:0001332); Dysarthria (HP:0001260); Drooling (HP:0002307); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347); excluded: Cerebral atrophy (HP:0002059); excluded: Cerebellar atrophy (HP:0001272)
Individual 5 (MALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.553A>G (heterozygous) NM_003086.4:c.2317+1G>T (heterozygous),Short stature (HP:0004322); Microcephaly (HP:0000252); Spasticity (HP:0001257); Dysarthria (HP:0001260); Drooling (HP:0002307); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347); excluded: Spastic tetraparesis (HP:0001285); excluded: Cerebral atrophy (HP:0002059); excluded: Cerebellar atrophy (HP:0001272)
Individual 6 (FEMALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.2428C>T (heterozygous) NM_003086.4:c.737+5G>T (heterozygous),P6M: Cognitive regression (HP:0034332); Motor regression (HP:0033044)Short stature (HP:0004322); Microcephaly (HP:0000252); Spasticity (HP:0001257); Dystonia (HP:0001332); Dysarthria (HP:0001260); Drooling (HP:0002307); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347); Cerebral atrophy (HP:0002059); Cerebellar atrophy (HP:0001272); excluded: Spastic tetraparesis (HP:0001285)
Individual 7 (FEMALE; ),"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)",NM_003086.4:c.2527+1G>A (homozygous),P1Y4M: Cognitive regression (HP:0034332); Motor regression (HP:0033044)Short stature (HP:0004322); Microcephaly (HP:0000252); Spastic tetraparesis (HP:0001285); Dysarthria (HP:0001260); Drooling (HP:0002307); Gait disturbance (HP:0001288); Hyperreflexia (HP:0001347); Cerebral atrophy (HP:0002059); Cerebellar

In [10]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 10 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [11]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())

[pyphetools] Ingested 10 GA4GH phenopackets.


In [12]:
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)

In [13]:
PMID = "PMID:36965478"
hpoa_table_creator = builder.autosomal_recessive(PMID).build()
df = hpoa_table_creator.get_dataframe()
df.head()

We found a total of 13 unique HPO terms
Extracted disease: Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction (OMIM:620515)


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",HP:0034332,Cognitive regression,,,8/8,,,,,PMID:36965478,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",HP:0033044,Motor regression,,,8/8,,,,,PMID:36965478,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",HP:0004322,Short stature,,,10/10,,,,,PMID:36965478,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",HP:0001257,Spasticity,,,4/4,,,,,PMID:36965478,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620515,"Neurodevelopmental disorder with motor regression, progressive spastic paraplegia, and oromotor dysfunction",HP:0001260,Dysarthria,,,8/10,,,,,PMID:36965478,PCS,ORCID:0000-0002-0736-9199


In [14]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620515.tab
